In [8]:
import pandas as pd
import json
from pathlib import Path
# Model Algorithms
from sklearn.manifold import TSNE
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [2]:
recipes = []
ingredients = []
ids = []
path = Path('~/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/')
for i, po in enumerate(path.expanduser().glob('*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        recipes.append(r)
        ing = r.get('Ingredients')
        id = r.get('RecipeID')
        ids.append(id)
        if ing is not None:
            ingredients.append([i['Name'] for i in ing if i is not None and i['Name'] is not None])
       


/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-19-156227.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-127-74-2431270.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-112-159-862538.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-62-166860.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-35-191-157959.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-74-82-565669.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-51-46-372714.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-108-181-2437430.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-13-123-101172.json
/Users/bridgitboulahanis/Dro

In [3]:

ing = pd.DataFrame(ingredients, index=ids)
ing
print(len(ingredients))
print(len([i for i in ingredients if None in i]))

ingstrings = [' '.join(i) for i in ingredients]
ingstrings

32526
0


['Garam masala Fresh coriander leaves; Fresh ginger root Turmeric Water Ground coriander Water Ground cumin red pepper; hot Rice Tea leaves Chana dal Salt Ghee Cumin seed Onion',
 'Pasta Shrimp Butter Fresh Italian parsley Red pepper flakes Ground black pepper  Kosher Salt Dry white wine garlic Parmesan cheese grated lemon',
 'Black Pepper Egg FOR HUSH PUPPIES whole grain Onion Salt Salt FOR THE FISH Buttermilk Baking soda Baking powder Corn flour Cayenne fish',
 'Chicken tenderloins or 4 chicken breast filets  sweet onion Honeu barbeque sauce  sugar  powdered garlic  black ground pepper  sea salt',
 'Chicken breasts boneless and skinless garlic Shallots minced Thai Chilies Ground white pepper Eggs Vegetable oil Soy sauce Thai basil leaves Fish sauce Granulated sugar',
 'Old El Paso Mild Enchilada Sauce Whole Wheat Tortillas Precooked Chicken Breast Shredded Reduced Fat Mexican Cheese Cumin Salt & Pepper',
 'Calypso Broth: Chicken Broth Clam juice Fresh garlic Blackened seasoning Cayen

In [4]:
## add block here with stem/lem and dedup party

In [5]:
vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
vect.fit(ingstrings)  
features = vect.transform(ingstrings)

cosine_similarities = linear_kernel(features[3:4], features).flatten()
related_docs_indices = cosine_similarities.argsort()[:-5:-1]
print(related_docs_indices)
print(cosine_similarities[related_docs_indices])
print("=" * 5)
for i in related_docs_indices: print(ingstrings[i])

[    3 28741 24293  3207]
[1.         0.49697508 0.41884586 0.3299788 ]
=====
Chicken tenderloins or 4 chicken breast filets  sweet onion Honeu barbeque sauce  sugar  powdered garlic  black ground pepper  sea salt
pork tenderloins garlic powder salt ground black pepper barbeque sauce
chicken  Barbeque sauce
Chicken breasts  Reduced-sodium soy sauce Sweet onion Powdered Ginger


In [6]:

pickle.dump(vect,open("bigoven/vect.pkl","wb"))
pickle.dump(features,open("bigoven/features.pkl","wb"))
rec = pd.DataFrame(recipes)
pickle.dump(rec,open("bigoven/recipes.pkl","wb"))



In [24]:
print(np.size(new_features))
print(np.size(features))
print(np.size(cosine_similarities))

33
634558
32526


In [266]:
print(rec.shape)


(32526, 46)

In [27]:
#proof of concept
a = pickle.load(open("bigoven/vect.pkl", 'rb'))
b = pickle.load(open("bigoven/features.pkl", 'rb'))
c =['2 skinless and boneless chicken breasts, butterflied and then cut in half Sea salt and freshly ground black pepper All-purpose flour, for dredging 6 tablespoons unsalted butter 5 tablespoons extra-virgin olive oil 1/3 cup fresh lemon juice 1/2 cup chicken stock 1/4 cup brined capers, rinsed 1/3 cup fresh parsley, chopped']

nf = a.transform(c)

cs = linear_kernel(nf, b).flatten()
rdi = cs.argsort()[:-5:-1]
print(rdi)
print(cs[rdi])
print("=" * 5)
for i in rdi: print(ingstrings[i])




[  641 23815  4049 22664]
[0.60407624 0.50858744 0.48633629 0.48289483]
=====
1/2 cup
Boneless, skinless chicken breasts Salt and pepper All purpose flour unsalted butter extra-virgin olive oil fresh lemon juice chicken stock brined capers fresh parsley
ingredients  1 1/4 cup red wine vinegar  1/4 cup s
chicken breasts Flour Extra virgin olive oil Unsalted butter Fresh lemon juice Chicken Stock brined capers fresh parsley Salt Black pepper


In [28]:
np.size(cs)

32526

In [218]:
# rec.iloc[28741]

RecipeID                                                          1174173
Title                                             Grilled Pork Tenderloin
Description             "This is one of my family's favorites. This po...
Cuisine                                                                  
Category                                                        Main Dish
Subcategory                                                 Grill and BBQ
Microcategory                                                        None
PrimaryIngredient                                                        
StarRating                                                              0
WebURL                  https://www.bigoven.com/recipe/grilled-pork-te...
ImageURL                http://redirect.bigoven.com/pics/rs/640/grille...
ReviewCount                                                             0
MedalCount                                                              0
FavoriteCount                         